In [18]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [19]:
#creates the training dataset
df_train = pd.read_csv("data/final/final_training.csv")
df_train = df_train[df_train['pot_doc'] != '[]']
X_train_df=pd.DataFrame(data = np.array(df_train), columns = df_train.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_train_df = pd.DataFrame(data = np.array(df_train['documentid']), columns = ['documentid'])

#creates the testing dataset
df_test = pd.read_csv("data/final/final_test.csv")
X_test_df=pd.DataFrame(data = np.array(df_test), columns = df_test.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_test_df = pd.DataFrame(data = np.array(df_test['documentid']), columns = ['documentid'])


In [20]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2)
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_test_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [21]:

def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    y = trial.suggest_float('y', 0, 10)
    z = trial.suggest_float('z', 0, 10)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,y,z,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}



[I 2023-01-29 09:47:24,847] A new study created in memory with name: no-name-508255be-8925-4dbf-82f0-5dc4ea3cbec8
[I 2023-01-29 09:47:25,225] Trial 0 finished with value: 0.1618122977346278 and parameters: {'x': 0.31707784364272684, 'y': 7.891919021076451, 'z': 4.346072564605821}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:47:25,607] Trial 1 finished with value: 0.1618122977346278 and parameters: {'x': 0.414655543308734, 'y': 8.559577172283683, 'z': 0.48286522735172643}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:47:25,977] Trial 2 finished with value: 0.1618122977346278 and parameters: {'x': 6.136318037764915, 'y': 6.032883167413786, 'z': 4.087992359275922}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:47:26,349] Trial 3 finished with value: 0.1618122977346278 and parameters: {'x': 0.6565401768253976, 'y': 8.14177423525944, 'z': 2.2124935610386154}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:47:26,72

{'x': 0.31707784364272684, 'y': 7.891919021076451, 'z': 4.346072564605821}

In [22]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['y'], 0)

real = list(y_test_df['documentid'])


In [23]:
'''
calculates the percentage where our predicted data conforms to our acutal data
'''
def calculate_percentage(list_a, list_b):
    common_elements = set(list_a) & set(list_b)
    percentage = (len(common_elements) / len(list_a)) * 100
    return percentage
    
#from our model, we have a predicted accuracy of 89.51612903225806
print(calculate_percentage(res,real))

89.51612903225806
